In [1]:
from main import *

In [2]:
signal_microphone, signal_loudspeaker, impulse_response, rate = generate_signals()

ValueError: object too deep for desired array

In [6]:
N = 1000

finish_index = 4 * rate # stop after 4 seconds
final_nlms_signal_error, estimated_impulse_response = NLMS(signal_loudspeaker, signal_microphone, N, 0.5, 0.000001, freeze_index=finish_index)
signal_error = final_nlms_signal_error


NameError: name 'rate' is not defined

In [5]:
plot_signals(signal_microphone, signal_loudspeaker, impulse_response, signal_error, estimated_impulse_response, N)

NameError: name 'signal_microphone' is not defined